## This notebook explores the relationship between the topics of Wikipedia pages edited by users and the occurrence of votes between two users. The goal is to identify potential correlations and patterns that would show that editing similar topics has an influence in the motivation to cast a vote.

In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
import gzip
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Download the dataset with all the votes
df = edits_df = pd.read_csv("../data/wiki-RfA-cleaned.csv")

#Set of users that are present in the adminship dataset
admin_set = set(df['source'].to_list() + df['target'].to_list())

In [3]:
# The original dataset can be found here (https://snap.stanford.edu/data/wiki-meta.html). 
# The version that we use here has already been modified so that we get each user and 
# the page they modified with the number of edits

edits_df = pd.read_csv("../data/interactions_edits_grouped.zip", index_col=0, compression='zip')
edits_df

,username,counts
article_title,,
!,MPerel,1
!!,Gracenotes,1
!!,Gwern,1
!!,JHunterJ,8
!!,Kane5187,1
...,...,...
ﻲ,Dbenbenn,1
ﻳ,Dbenbenn,1
ﻴ,Dbenbenn,1


In [4]:
# We create a list of Wikipedia pages modified by each users
user_indices = edits_df.groupby('username').apply(lambda x: x.index.tolist()).reset_index(name='Indices')
user_indices

,username,Indices
0,'sed,"[Bloody_Mary_(South_Park), Cartoon_Wars_Part_I..."
1,(.Y.),"[Blue_Water_High, List_of_bass_guitarists, Lis..."
2,(:Julien:),"[1993_Russian_constitutional_crisis, 1999_NBA_..."
3,(aeropagitica),"['Aho'eitu, 'Ilaheva, ...Und_Morgen_Fliegen_Wi..."
4,*Kat*,"[&_Yet_&_Yet, *MTV, .nc.tr, 1670, 1964_(emulat..."
...,...,...
6586,^demon,"[""Comedy_Time""_Ganesh, ""Them""_(King_Diamond_al..."
6587,~K,"[(Bis(trifluoroacetoxy)iodo)benzene, 1,2,4-Tri..."
6588,Вasil,"[1809, 1870, 1877, 1964_Minnesota_Twins_season..."
6589,Еdit,"[1600_Pennsylvania_Avenue_(musical), 2005_Atla..."


In [5]:
# Create a list of all users present in the edits dataset
users = set(edits_df['username'].tolist())

#### Create a matrix with the Jaccard index (on the the lists of modified pages) for all pairs of users

In [6]:
matrix_similarity = pd.read_csv("../data/jaccard.csv.zip", index_col=0, compression='zip')
matrix_similarity

,Bellhalla,Complainer,Werdan7,Grön,Weasel,Filll,Thejerm,Cyclone1,RJFJR,Mkweise,...,UtherSRG,Pyrop,AngelOfSadness,Clarkk,Appleboy,Voyaging,JLaTondre,InShaneee,Ed,Advanced
Bellhalla,1.000000,0.000173,0.000808,0.000000,0.000000,0.000579,0.000315,0.000173,0.001307,0.000426,...,0.001782,0.000296,0.001437,0.000543,0.000334,0.000000,0.004429,0.001600,0.000168,0.000000
Complainer,0.000173,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000492,0.000000,0.000369,0.000000,0.000000,0.000000,0.000098,0.000000,0.000000,0.000000
Werdan7,0.000808,0.000000,1.000000,0.000000,0.000000,0.001804,0.000000,0.000000,0.000596,0.002238,...,0.001802,0.002651,0.008425,0.000504,0.009511,0.002049,0.002832,0.006304,0.004392,0.005298
Grön,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000104,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.003106,0.000000,0.000196,0.000324,0.000000,0.000000
Weasel,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000246,0.000000,0.000000,0.000000,0.000000,0.000000,0.000195,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Voyaging,0.000000,0.000000,0.002049,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.003484,1.000000,0.000197,0.000327,0.004348,0.000000
JLaTondre,0.004429,0.000098,0.002832,0.000196,0.000195,0.001678,0.002697,0.000000,0.005550,0.004555,...,0.008998,0.002688,0.009551,0.002571,0.002116,0.000197,1.000000,0.011813,0.000676,0.000959
InShaneee,0.001600,0.000000,0.006304,0.000324,0.000000,0.004031,0.003552,0.000644,0.005090,0.007861,...,0.007856,0.006660,0.014829,0.001686,0.009451,0.000327,0.011813,1.000000,0.001847,0.001505
Ed,0.000168,0.000000,0.004392,0.000000,0.000000,0.000711,0.000000,0.000000,0.000408,0.001306,...,0.000549,0.001596,0.001755,0.000270,0.006224,0.004348,0.000676,0.001847,1.000000,0.002000


Create a DataFrame with all pairs of users and a binary variable that indicates if a vote exists for each pair.

In [7]:
similarity_and_vote = pd.read_csv("../data/jaccard_and_votes.csv.zip", index_col=0, compression='zip')
similarity_and_vote

,vote,jaccard
"('Bellhalla', 'Complainer')",0,0.000173
"('Bellhalla', 'Werdan7')",0,0.000808
"('Bellhalla', 'Grön')",0,0.000000
"('Bellhalla', 'Weasel')",0,0.000000
"('Bellhalla', 'Filll')",0,0.000579
...,...,...
"('JLaTondre', 'Ed')",0,0.000676
"('JLaTondre', 'Advanced')",0,0.000959
"('InShaneee', 'Ed')",0,0.001847
"('InShaneee', 'Advanced')",0,0.001505


### Now we compute some statistics on this relation.

In [8]:
pearsonr(similarity_and_vote['vote'], similarity_and_vote['jaccard'])

PearsonRResult(statistic=0.08914982798437746, pvalue=0.0)

The correlation between similarity score on edited pages and the votes is not very strong but positive with high significance. 

In [9]:
# Mean similarity between all pairs of users
mean_sim_all = similarity_and_vote['jaccard'].mean()
mean_sim_all

0.0012019944311329655

In [10]:
# Mean similarity between pairs of users that are linked by a vote
mean_sim_vote = similarity_and_vote[similarity_and_vote['vote'] == 1]['jaccard'].mean()
mean_sim_vote

0.005487495987236598

In [11]:
print(f"People that are linked by a vote have {mean_sim_vote / mean_sim_all:.2f} times more common edited pages than the average.")

People that are linked by a vote have 4.57 times more common edited pages than the average.


##### Now we investigate the most edited pages of the communities

In [12]:
community5 = {'*Spark*',
 'AaronS',
 'Afaprof01',
 'Alanyst',
 'Alasdair',
 'Amgine',
 'Anser',
 'Antelan',
 'Anthon01',
 'Archola',
 'Avb',
 'Averykrouse',
 'BRMo',
 'Badgerpatrol',
 'Baegis',
 'Banno',
 'BanyanTree',
 'Barek',
 'Bengalski',
 'BernardL',
 'BlueMoonlet',
 'BoNoMoJo',
 'Brusegadi',
 'COGDEN',
 'CTSWyneken',
 'Cardamon',
 'Cjewell',
 'ConfuciusOrnis',
 'Cookiecaper',
 'Cornerbock',
 'CrazyInSane',
 'Crohnie',
 'Dabomb87',
 'Diego',
 'Dlabtot',
 'Doright',
 'Drboisclair',
 'Duke53',
 'EffK',
 'ElectricRay',
 'FCYTravis',
 'FT2',
 'FeloniousMonk',
 'Filll',
 'Flex',
 'Frelke',
 'Friarslantern',
 'Fyslee',
 'Gandalfxviv',
 'Giovanni33',
 'Gkhan',
 'Gleng',
 'Gwynand',
 'Haber',
 'Harpchad',
 'Hawstom',
 'Homestarmy',
 'Infinity0',
 'Iuio',
 'JedRothwell',
 'Jefffire',
 'Jim62sch',
 'Jimmyvanthach',
 'Jimp',
 'Jmh123',
 'Jwanders',
 'KHM03',
 'Kasreyn',
 'Kenosis',
 'KimvdLinde',
 'Knucmo2',
 'Kokota',
 'L0b0t',
 'Lethargy',
 'Levine2112',
 'Lovecoconuts',
 'LuckyLouie',
 'Mantanmoreland',
 'Mariordo',
 'MaxPont',
 'Mccready',
 'Megapixie',
 'Merzul',
 'Michaelsanders',
 'Mkmcconn',
 'MonkeeSage',
 'Morphh',
 'Nikodemos',
 'Noleander',
 'Orangemarlin',
 'PatrickA',
 'Paulmcdonald',
 'PouponOnToast',
 'QuackGuru',
 'RJII',
 'Ramdrake',
 'Reaverdrop',
 'Rednblu',
 'Reinis',
 'Reswobslc',
 'Ronabop',
 'Rosicrucian',
 'SJK',
 'SOPHIA',
 'SamEV',
 'Samiharris',
 'ScienceApologist',
 'Slrubenstein',
 'Snalwibma',
 'Spottedowl',
 'StAnselm',
 'StanZegel',
 'SteveRwanda',
 'Str1977',
 'Sugaar',
 'Taxwoman',
 'Tevildo',
 'TheNautilus',
 'Thedreamdied',
 'Thetruthbelow',
 'Timelessness',
 'Tisthammerw',
 'Trevdna',
 'Trödel',
 'Ultramarine',
 'Ungtss',
 'Visorstuff',
 'WBardwin',
 'Wdanwatts',
 'Wesley',
 'WilliamThweatt',
 'Winklethorpe',
 'Wistless',
 'Wobble',
 'Wrp103',
 'Xiutwel',
 'Zero1328',
 '^^James^^'}

community12 = {'999',
 'AMbroodEY',
 'Abecedare',
 'Aervanath',
 'Akut',
 'Altruism',
 'Ambuj.Saxena',
 'Anmol.2k4',
 'Ansell',
 'Anupam',
 'AreJay',
 'Armanaziz',
 'Arunram',
 'Ascidian',
 'Autumnleaf',
 'Bakasuprman',
 'Bhadani',
 'BhaiSaab',
 'Bharatveer',
 'BigHaz',
 'BlackJack',
 'Blacksun',
 'Blnguyen',
 'Bondego',
 'BostonMA',
 'Boven',
 'Calabraxthis',
 'Canaen',
 'Chicocvenancio',
 'Clt13',
 'Cribananda',
 'Crocogator',
 'Crownofworms',
 'DRK',
 'DaGizza',
 'Dafyd',
 'Dagadt',
 'Dangerous-Boy',
 'Daniel11',
 'Deeptrivia',
 'Deville',
 'Dineshkannambadi',
 'Dlodge',
 'Doctorbruno',
 'DrunkenSmurf',
 'Dwaipayanc',
 'Dweller',
 'Eukesh',
 'Evadb',
 'EyeMD',
 'FRS',
 'Finetooth',
 'Fowler&fowler',
 'Ganeshk',
 'GenghizRat',
 'Geo.plrd',
 'Gnanapiti',
 'Gsingh',
 'Gurubrahma',
 'Guycalledryan',
 'Happy-melon',
 'Harikishore',
 'Hornplease',
 'HussainAbbas',
 'Idleguy',
 'ImpalerBugz',
 'ImpuMozhi',
 'Indiver',
 'Jayanthv86',
 'Jayen466',
 'Jftsang',
 'Jishacj',
 'Jjjsixsix',
 'Jodmar',
 'Johnlp',
 'Jpeob',
 'KNM',
 'Kefalonia',
 'Leotolstoy',
 'Loganberry',
 'Loom91',
 'LordViD',
 'Lostintherush',
 'Lustead',
 'Magicalsaumy',
 'Masterbobo',
 'Mattisse',
 'MaximvsDecimvs',
 'Mdmanser',
 'Mereda',
 'MichelleG',
 'Miljoshi',
 'MinaretDk',
 'Myciconia',
 'Nathan256',
 'Natkeeran',
 'Naveenbm',
 'Naveenpf',
 'Ncmvocalist',
 'Nephron',
 'Nichalp',
 'NinaOdell',
 'Nips',
 'Nobleeagle',
 'NubKnacker',
 'P.K.Niyogi',
 'Pamri',
 'Peripatetic',
 'Phronima',
 'Pigman',
 'Polaris999',
 'Premkudva',
 'Priyanath',
 'Qaanaaq',
 'Rabbeinu',
 'Raghu.kuttan',
 'Ragib',
 'Rajithmohan',
 'Ravichandar84',
 'Rayfield',
 'Redtigerxyz',
 'Rrjanbiah',
 'SBC-YPR',
 'Saksham',
 'Samir',
 'Santhoshguru',
 'Saravask',
 'Sarvagnya',
 'Sastrawan',
 'Sdedeo',
 'Sgrayban',
 'Shmitra',
 'Shreevatsa',
 'Shreshth91',
 'Shushruth',
 'Shyamsunder',
 'SilkTork',
 'Sisodia',
 'Sophitus',
 'Spangineer',
 'Spartiate',
 'Spasage',
 'Spasemunki',
 'Srikeit',
 'Stevenmitchell',
 'Stlemur',
 'Sukh',
 'Sumalsn',
 'Sundar',
 'Supten',
 'SynergeticMaggot',
 'THUGCHILDz',
 'Taprobanus',
 'Taxman',
 'TheOtherBob',
 'Thenub314',
 'Tintin1107',
 'Utcursch',
 'Vadakkan',
 'Vivin',
 'Vyzasatya',
 'Watchdogb',
 'WikiScholarChad',
 'Wikiality123',
 'Willscrlt',
 'Wisesabre',
 'Xdamr',
 'Yessrao',
 'Zaheer89',
 'Zamkudi',
 'Zephyr2k',
 'Zleitzen'}

community10 = {'Ahunt',
 'Akradecki',
 'Archtransit',
 'Bellhalla',
 'BillCJ',
 'Brad101',
 'Bzuk',
 'DAJF',
 'Emt147',
 'Ericg',
 'Fnlayson',
 'Gatoclass',
 'Greglocock',
 'Guapovia',
 'Guest9999',
 'Jareand',
 'Jmount',
 'Judae1',
 'Jwillbur',
 'Kralizec!',
 'Kubek15',
 'Kwh',
 'Lawlore',
 'MBK004',
 'Manxruler',
 'Maralia',
 'Mario1987',
 'MilborneOne',
 'Mister1nothing',
 'MoRsE',
 'Mrbowtie',
 'NAHID',
 'Neil',
 'Nimbus227',
 'Orpy15',
 'Parsecboy',
 'Piemanmoo',
 'Raza0007',
 'Red4tribe',
 'Rjwilmsi',
 'Rlandmann',
 'SG',
 'Shibumi2',
 'Smuuv',
 'Spahbod',
 'Squalla',
 'TabooTikiGod',
 'TestPilot',
 'Thatguy96',
 'Thunderhead1',
 'TomStar81',
 'TomTheHand'}

In [13]:
community_to_check = community10

In [14]:
ul = list(user_indices['username'])
all_subject = set()
for ind in community_to_check:
    if ind in ul:
        all_subject = set(user_indices[user_indices['username'] == ind]['Indices'].iloc[0]).union(all_subject)

In [15]:
data = pd.DataFrame(index=list(all_subject), columns=['count'])
data = data.fillna(0)

In [16]:
ul = list(user_indices['username'])
for ind in community_to_check:
    if ind in ul:
        a = user_indices[user_indices['username'] == ind]['Indices'].iloc[0]
        for s in a:
            data.at[s, 'count'] += 1

In [17]:
data.sort_values('count', ascending = False).head(50)

,count
F-104_Starfighter,12
F-16_Fighting_Falcon,12
F-22_Raptor,12
Messerschmitt_Me_262,12
Messerschmitt_Bf_109,11
B-24_Liberator,11
B-25_Mitchell,11
De_Havilland_Mosquito,11
P-51_Mustang,11
Boeing_767,11
